In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from ExplicitModels import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.1306,), std=(0.3081,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
# # activation = torch.sigmoid
# def mysoftmax(x, dim = 0):
#     return F.softmax(x, dim)

# activation = F.relu

# architecture = [784, 128, 64, 10]
# lambda_ = 0.9999
# epsilon = 0.01
# neural_lr_start = 0.1
# neural_lr_stop = 0.00
# neural_lr_rule = "divide_by_slow_loop_index"
# neural_lr_decay_multiplier = 0.005
# neural_dynamic_iterations = 50

# model = CorInfoMaxBiDirectional_wAutoGrad(architecture, lambda_, epsilon, activation, optimizer_type = "sgd",
#                                           optim_lr_ff = 0.05, optim_lr_fb = 0.005, use_stepLR = True, 
#                                           stepLR_step_size = 10*3000)

# # activation = torch.sigmoid
# def mysoftmax(x, dim = 0):
#     return F.softmax(x, dim)

# activation = F.relu

# architecture = [32*32*3, 1000, 10]
# lambda_ = 0.9999
# epsilon = 0.01
# neural_lr_start = 0.1
# neural_lr_stop = 0.00
# neural_lr_rule = "divide_by_slow_loop_index"
# neural_lr_decay_multiplier = 0.005
# neural_dynamic_iterations = 50

# model = CorInfoMaxBiDirectional_wAutoGrad(architecture, lambda_, epsilon, activation, optimizer_type = "sgd",
#                                           optim_lr_ff = 0.01, optim_lr_fb = 0.001, use_stepLR = True, 
#                                           stepLR_step_size = 10*3000)

# activation = torch.sigmoid
def mysoftmax(x, dim = 0):
    return F.softmax(x, dim)

def kWTA(inp_vector, k = 1):
    tmpx = torch.clone(inp_vector)
    topval = tmpx.topk(k, dim=0)[0][k - 1]
    topval = topval.repeat(tmpx.shape[0], 1)
    comp = (inp_vector>=topval)
    return comp*inp_vector

def Sparsify_kWTA(inp_vector, sparsity = 0.1):
    k = int(inp_vector.shape[0] * sparsity)
    if k == 0:
        k = 1
    tmpx = torch.clone(inp_vector)
    topval = tmpx.topk(k, dim=0)[0][k - 1]
    topval = topval.repeat(tmpx.shape[0], 1)
    comp = (inp_vector>=topval)
    return comp*inp_vector

activation = Sparsify_kWTA
activation = torch.nn.LeakyReLU(0.0)
activation = torch.sigmoid

architecture = [784, 500, 10]
lambda_ = 0.9999
epsilon = 0.01
neural_lr_start = 0.1
neural_lr_stop = 0.00
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.005
neural_dynamic_iterations = 50

model = CorInfoMaxBiDirectional_wAutoGrad(architecture, lambda_, epsilon, activation, optimizer_type = "sgd",
                                          optim_lr_ff = 0.01, optim_lr_fb = 0.0001, 
                                          use_stepLR = True, sgd_weight_decay = 0.001, 
                                          stepLR_step_size = 10*3000)

In [5]:
# x, y = next(iter(train_loader))
# x = x.view(x.size(0),-1).T
# x, y = x.to(device), y.to(device)
# y = F.one_hot(y, 10).to(device).T
# neurons = model.init_neurons(x.size(1))
# # neurons = model.fast_forward(x)
# mode = "train"
# if mode == "train":
#     neurons[-1] = y.to(torch.float)


# neurons = model.run_neural_dynamics( x, y, neurons, neural_lr_start, neural_lr_stop, lr_rule = neural_lr_rule,
#                                     lr_decay_multiplier = neural_lr_decay_multiplier, 
#                                     neural_dynamic_iterations = neural_dynamic_iterations)

In [6]:
# plt.plot(model.neural_dynamics_loss_list)

In [7]:
# F.softmax(neurons[0], 0)[:,0]

In [8]:
# torch.sum(F.softmax(neurons[0], 0), 0)

In [9]:
# optim_params = []
# for idx in range(len(model.Wff)):
#     for key_ in ["weight", "bias"]:
#         optim_params.append(  {'params': model.Wff[idx][key_], 'lr': lr_start["ff"]}  )

In [10]:
# optimizer = torch.optim.Adam(optim_params, maximize = True)

In [11]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50

for epoch_ in range(n_epochs):
    if epoch_ > 12:
        neural_lr_start = 0.05
    if epoch_ > 17:
        neural_lr_start = 0.03
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        #x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid")
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        #y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        y_one_hot = 1*y_one_hot
        
        model.batch_step(  x, y_one_hot, neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                         )

    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [02:43, 18.32it/s]
2it [00:00, 14.30it/s]

Epoch : 1, Train Accuracy : 0.9018833333333334, Test Accuracy : 0.9044


3000it [02:42, 18.43it/s]
2it [00:00, 14.32it/s]

Epoch : 2, Train Accuracy : 0.91705, Test Accuracy : 0.9188


3000it [02:43, 18.40it/s]
2it [00:00, 14.32it/s]

...

Epoch : 48, Train Accuracy : 0.09871666666666666, Test Accuracy : 0.098


3000it [02:43, 18.33it/s]
2it [00:00, 13.81it/s]

Epoch : 49, Train Accuracy : 0.09871666666666666, Test Accuracy : 0.098


3000it [02:43, 18.33it/s]


Epoch : 50, Train Accuracy : 0.09871666666666666, Test Accuracy : 0.098


In [ ]:
# def inverse_leaky_relu(x, negative_slope = 0.1):
#     out = torch.zeros(*x.shape)
#     out[x >]

In [ ]:
Wfb[1]['weight'] @ yy + Wfb[1]['bias']

In [ ]:
torch.clip(Wfb[1]['weight'] @ yy + Wfb[1]['bias'], 1e-3, 1-1e-3)

In [ ]:
model.Wfb[1]

In [ ]:
cc = 0
yy = torch.randn(10, 1).to(device)
yy[cc] = 4
Wfb = model.Wfb
out = torch.clip(Wfb[0]['weight'] @ activation_inverse(torch.clip(Wfb[1]['weight'] @ yy + Wfb[1]['bias'], 1e-3, 1-1e-3), 'sigmoid') + Wfb[0]['bias'], 1e-3, 1-1e-3).view(28 ,28)
out = torch2numpy(out)
plt.imshow(out)

In [ ]:
def accuracy_topk(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].flatten().float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
        
    result = [res_.item() for res_ in res]
    if len(result) == 1:
        result = result[0]
    return result

def evaluatePC_topk(model, loader, device, apply_activation_inverse = True, 
                    activation_type = "sigmoid", topk = [2], printing = True):
    # Evaluate Predictive Coding Model on Classification Task
    correct=0
    phase = 'Train' if loader.dataset.train else 'Test'
    
    for x, y in loader:
        if apply_activation_inverse:
            x = activation_inverse(x.view(x.size(0),-1).T, activation_type).to(device)
        else:
            x = x.view(x.size(0),-1).T.to(device)
        batch_size = x.size(1)
        y = y.to(device)
        
        neurons = model.fast_forward(x)

        #pred = torch.argmax(neurons[-1], dim=0).squeeze()  
        pred = neurons[-1]
        correct += accuracy_topk(pred.T, y, topk) * batch_size

    acc = correct/len(loader.dataset) 
    if printing:
        print(phase+' accuracy :\t', acc)   
    return acc

trn_acc = evaluatePC_topk(model, train_loader, device, False, 
                     printing = True, topk = [2])
tst_acc = evaluatePC_topk(model, test_loader, device, False, 
                     printing = True, topk = [2])

In [ ]:
for g in model.optimizer.param_groups:
    g['lr'] = g['lr']/100

In [ ]:


for epoch_ in range(n_epochs):
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        #x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid")
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        
        model.batch_step(  x, y_one_hot, neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                         )

    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))